# All Policies

Analyze the performance of our Whittle and Adaptive Policies

In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import sys
sys.path.append('/usr0/home/naveenr/projects/food_rescue_preferences')

In [8]:
import numpy as np
import random 
import matplotlib.pyplot as plt
import json 
import argparse 
import sys
from copy import deepcopy
import secrets
from itertools import combinations
from rmab.simulator import create_transitions_from_prob
from rmab.compute_whittle import arm_compute_whittle_multi_state, arm_compute_whittle

In [17]:
n_arms = 3
budget = 1

probabilities = [np.random.random() for i in range(n_arms)]

In [18]:
def total_prob(action):
    action = np.array(action)
    if np.sum(action) == 0:
        return 0
    return 1/np.sum(action)

In [19]:
all_transitions,probs_by_partition = create_transitions_from_prob("food_rescue",43,max_transition_prob=0.25)

In [22]:
def create_transition(prob):
    transitions = np.zeros((2,2,2))
    transitions[:,0,0] = 1
    transitions[:,1,1] = prob 
    transitions[:,1,0] = 1-prob 
    return transitions

In [24]:
create_transition(0.5)

array([[[1. , 0. ],
        [0.5, 0.5]],

       [[1. , 0. ],
        [0.5, 0.5]]])

In [25]:
state = [0 for i in range(n_arms)]

In [27]:
prob_list_default = [0.5,0.5,0.52]
prob_multiplier_001 = 1
prob_multiplier_010 = 1
prob_multiplier_100 = 1.5

In [26]:
def get_transitions(global_action,seed):
    np.random.seed(seed)    
    transitions = np.array([create_transition(i) for i in prob_list_default])

    if list(global_action) == [1,0,0]:
        transitions = np.array([create_transition(i*prob_multiplier_100) for i in prob_list_default])

    return transitions

In [14]:
seed = 44

In [29]:
start_action = np.zeros((n_arms))
action = np.zeros((n_arms)) 
start_action[np.random.choice(list(range(n_arms)),size=budget,replace=False)] = 1

while False in (action == start_action):
    print(action,start_action)
    state_WI = []

    transitions = get_transitions(start_action,seed)
    for i in range(n_arms):
        state_WI.append(arm_compute_whittle(transitions[i,:,:,1], state[i], 0.9, 0, eps=1e-3,reward_function='combined',lamb=1,match_prob=0,match_probability_list=[],get_v=False,num_arms=n_arms))
    print(state_WI)
    selected_idx = np.argsort(state_WI)[-budget:][::-1]
    action = deepcopy(start_action)
    start_action = np.zeros(n_arms, dtype=np.int8)
    start_action[selected_idx] = 1
    print("Best response {}".format(start_action))

[0. 0. 0.] [0. 0. 1.]
[0.15020370483398438, 0.15020370483398438, 0.15592575073242188]
Best response [0 0 1]


In [30]:
max_whittle_sum = 0
max_action = []

for i in range(2**n_arms):
    binary_rep = bin(i)[2:]
    action = [int(j) for j in binary_rep.zfill(n_arms)]
    if np.sum(action) <= budget:
        transitions = get_transitions(action,seed)
        state_WI = []
        for j in range(n_arms):
            if action[j] == 1:
                state_WI.append(arm_compute_whittle(transitions[j,:,:,1], state[j], 0.9, 0, eps=1e-3,reward_function='combined',lamb=0.5,match_prob=0.5,match_probability_list=[],get_v=False,num_arms=n_arms))
        if np.sum(state_WI) > max_whittle_sum:
            max_action = action
            max_whittle_sum = np.sum(np.array(state_WI))
max_action

[1, 0, 0]